# Code Interpreter with Azure AI Agent Service 

## Import Packages 

In [ ]:
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import CodeInterpreterTool
from azure.identity import DefaultAzureCredential
from typing import Any
from pathlib import Path
from pprint import pformat
from datetime import datetime

from tabulate import tabulate

## Connecct to AI Foundry Project 

In [17]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

In [18]:
with project_client:
    # Create an instance of the CodeInterpreterTool
    code_interpreter = CodeInterpreterTool()

    # The CodeInterpreterTool needs to be included in creation of the agent
    agent = project_client.agents.create_agent(
        model="gpt-4o-mini",
        name="my-agent",
        instructions="You are helpful agent",
        tools=code_interpreter.definitions,
        tool_resources=code_interpreter.resources,
    )
    print(f"Created agent, agent ID: {agent.id}")

    # Create a thread
    thread = project_client.agents.create_thread()
    print(f"Created thread, thread ID: {thread.id}")

    # Create a message
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="Could you please create a bar chart for the operating profit using the following data and provide the file to me? Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, Company D: $1.8 million",
    )
    print(f"Created message, message ID: {message.id}")

    # Run the agent
    run = project_client.agents.create_and_process_run(
        thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        # Check if you got "Rate limit is exceeded.", then you want to get more quota
        print(f"Run failed: {run.last_error}")

    # Get messages from the thread
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print("\n=== Thread Messages ===")
    print(pformat(messages.__dict__, indent=2, width=80))

    # Get the last message from the sender
    # last_msg = messages.get_last_text_message_by_sender("assistant")
    # if last_msg:
    #     print(f"Last Message: {last_msg.text.value}")




    # Generate an image file for the bar chart
    for image_content in messages.image_contents:
        print(f"Image File ID: {image_content.image_file.file_id}")
        file_name = f"{image_content.image_file.file_id}_image_file.png"
        project_client.agents.save_file(
            file_id=image_content.image_file.file_id, file_name=file_name)
        print(f"Saved image file to: {Path.cwd() / file_name}")

    # Print the file path(s) from the messages
    for file_path_annotation in messages.file_path_annotations:
        print(f"File Paths:")
        print(f"Type: {file_path_annotation.type}")
        print(f"Text: {file_path_annotation.text}")
        print(f"File ID: {file_path_annotation.file_path.file_id}")
        print(f"Start Index: {file_path_annotation.start_index}")
        print(f"End Index: {file_path_annotation.end_index}")
        project_client.agents.save_file(
            file_id=file_path_annotation.file_path.file_id, file_name=Path(file_path_annotation.text).name)

    # Delete the agent once done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

Created agent, agent ID: asst_F7iWSLR09ZnwcOtRswYAdre1
Created thread, thread ID: thread_ZHOdw0NZ0ZPNdhF953dFOGff
Created message, message ID: msg_uK2UzRxZc0T00Pyv45uazJiu
Run finished with status: RunStatus.COMPLETED

=== Thread Messages ===
{ '_data': { 'data': [ {'id': 'msg_HymGxkLcpQANdr6iPOb9ZiXV', 'object': 'thread.message', 'created_at': 1740151044, 'assistant_id': 'asst_F7iWSLR09ZnwcOtRswYAdre1', 'thread_id': 'thread_ZHOdw0NZ0ZPNdhF953dFOGff', 'run_id': 'run_mXZFTQ0p2XJRjxyPWQty9OKX', 'role': 'assistant', 'content': [{'type': 'image_file', 'image_file': {'file_id': 'assistant-YN722cEvt4QG9gHsVEbN4a'}}, {'type': 'text', 'text': {'value': 'I have created the bar chart for the operating profits of the companies. You can download it using the link below:\n\n[Download the Operating Profit Chart](sandbox:/mnt/data/operating_profit_chart.png)', 'annotations': [{'type': 'file_path', 'text': 'sandbox:/mnt/data/operating_profit_chart.png', 'start_index': 154, 'end_index': 198, 'file_path